# An-Cockrell model reimplementation



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import itertools
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display
import h5py
from enum import IntEnum
from tqdm.notebook import tqdm

import an_cockrell
from an_cockrell import EpiType

In [ ]:
%matplotlib inline

In [ ]:
model = an_cockrell.AnCockrellModel(
    is_bat=False,
    init_inoculum=100,
    init_dcs=50,
    init_nks=25,
    init_macros=50,
    macro_phago_recovery=0.5,
    macro_phago_limit=1_000,
    inflammasome_activation_threshold=10,  # default 50 for bats
    inflammasome_priming_threshold=1.0,  # default 5.0 for bats
    viral_carrying_capacity=500,
    susceptibility_to_infection=77,
    human_endo_activation=5,
    bat_endo_activation=10,
    bat_metabolic_byproduct=2.0,
    human_metabolic_byproduct=0.2,
    # NOT USED resistance_to_infection=75,
    viral_incubation_threshold=60,
    # discovered parameters
    epi_apoptosis_threshold_lower=450,
    epi_apoptosis_threshold_range=100,
    epi_apoptosis_threshold_lower_regrow=475,
    epi_apoptosis_threshold_range_regrow=51,
    epi_regrowth_counter_threshold=432,
    epi_cell_membrane_init_lower=975,
    epi_cell_membrane_init_range=51,
    infected_epithelium_ros_damage_counter_threshold=10,
    epithelium_ros_damage_counter_threshold=2,
    epithelium_pdamps_secretion_on_death=10,
    dead_epithelium_pdamps_burst_secretion=10,
    dead_epithelium_pdamps_secretion=1,
    epi_max_tnf_uptake=0.1,
    epi_max_il1_uptake=0.1,
    epi_t1ifn_secretion=0.75,
    epi_t1ifn_secretion_prob=0.01,
    epi_pdamps_secretion_prob=0.01,
    infected_epi_t1ifn_secretion=1.0,
    infected_epi_il18_secretion=0.11,
    infected_epi_il6_secretion=0.10,
    activated_endo_death_threshold=0.5,
    activated_endo_adhesion_threshold=36,
    activated_endo_pmn_spawn_prob=0.1,
    activated_endo_pmn_spawn_dist=5,
    bat_t1ifn_init_amount=5.0,
    bat_t1ifn_init_prob=0.01,
    extracellular_virus_init_amount_lower=80,
    extracellular_virus_init_amount_range=40,
    bat_viral_lower_bound=1.0,
    human_viral_lower_bound=0.0,
    bat_t1ifn_effect_scale=0.1,
    human_t1ifn_effect_scale=0.01,
    pmn_max_age=36,
    pmn_ros_secretion_on_death=10.0,
    pmn_il1_secretion_on_death=1.0,
    nk_ifng_secretion=1.0,
    macro_max_virus_uptake=10.0,
    macro_activation_threshold=5.0,
    activated_macro_il8_secretion=1.0,
    activated_macro_il12_secretion=0.5,
    activated_macro_tnf_secretion=1.0,
    activated_macro_il6_secretion=0.4,
    activated_macro_il10_secretion=1.0,
    macro_antiactivation_threshold=-5.0,
    antiactivated_macro_il10_secretion=0.5,
    inflammasome_il1_secretion=1.0,
    inflammasome_macro_pre_il1_secretion=5.0,
    inflammasome_il18_secretion=1.0,
    inflammasome_macro_pre_il18_secretion=0.5,
    pyroptosis_macro_pdamps_secretion=10.0,
    dc_t1ifn_activation_threshold=1.0,
    dc_il12_secretion=0.5,
    dc_ifng_secretion=0.5,
    dc_il6_secretion=0.4,
    dc_il6_max_uptake=0.1,
    extracellular_virus_diffusion_const=0.05,
    T1IFN_diffusion_const=0.1,
    PAF_diffusion_const=0.1,
    ROS_diffusion_const=0.1,
    P_DAMPS_diffusion_const=0.1,
    IFNg_diffusion_const=0.2,
    TNF_diffusion_const=0.2,
    IL6_diffusion_const=0.2,
    IL1_diffusion_const=0.2,
    IL10_diffusion_const=0.2,
    IL12_diffusion_const=0.2,
    IL18_diffusion_const=0.2,
    IL8_diffusion_const=0.3,
    extracellular_virus_cleanup_threshold=0.05,
    cleanup_threshold=0.1,
    evap_const_1=0.99,
    evap_const_2=0.9,
    # non-dynamic variables
    GRID_WIDTH=51,
    GRID_HEIGHT=51,
    MAX_DCS=50,  # DCs are constant, so == init_dcs
    MAX_NKS=25,  # NKs are constant, so == init_nks
    MAX_MACROPHAGES=100,  # number of macrophages is constant, but are born/die. So we don't need a big buffer
    # MAX_PMNS=1000,  # keep default
    HARD_BOUND=True, # do not resize agent arrays if max is exceeded
)

* Blue Squares = Healthy Epithelial Cells
* Yellow Squares = Infected Epithelial Cells
* Grey Squares = Epithelial Cells killed by necrosis
* Grey Pentagons = Epithelial Cells killed by apoptosis
* Green Circles = Macrophages
* Large Green Circles = Macrophages at phagocytosis limit
* Orange Circles = NK Cells
* Light Blue Triangles = Dendritic Cells
* Pink Square Outlines = Activated Endothelial Cells
* Small White Circles = PMNs

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 8))

model.plot_agents(axs[0])
model.plot_field(axs[1], field_name="extracellular_virus")

axs[0].set_aspect(1)
axs[1].set_aspect(1)

In [ ]:
# model.infect(
#     init_inoculum=100
# )  # Initial-inoculum from 25-150 increments of 25, run for 14 days (2016 steps)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 8))

model.plot_agents(axs[0])
model.plot_field(axs[1], field_name="extracellular_virus")


axs[0].set_aspect(1)
axs[1].set_aspect(1)

In [ ]:
num_steps = 2016  # <- full run value


class Task(IntEnum):
    html_anim = 0
    save_mpeg = 1
    make_hdf5 = 2


task = Task.html_anim

In [ ]:
fields = [
    "extracellular_virus",
    "P_DAMPS",
    "ROS",
    "PAF",
    "TNF",
    "IL1",
    "IL6",
    "IL8",
    "IL10",
    "IL12",
    "IL18",
    "IFNg",
    "T1IFN",
]

total_T1IFN = []
total_TNF = []
total_IFNg = []
total_IL6 = []
total_IL1 = []
total_IL8 = []
total_IL10 = []
total_IL12 = []
total_IL18 = []
total_extracellular_virus = []
total_intracellular_virus = []
apoptosis_eaten_counter = []
infected_epis = []
dead_epis = []
apoptosed_epis = []


pbar = tqdm(total=num_steps, display=False)

In [ ]:
if task in {Task.html_anim, Task.save_mpeg}:
    fig = plt.figure(layout="constrained", figsize=(16, 8))
    fig.set_tight_layout(True)

    subfigs = fig.subfigures(1, 2, wspace=0.07)

    ax = subfigs[0].add_subplot()
    axs = subfigs[1].subplots(4, 4)

    def animate(i):
        model.time_step()
        pbar.update()

        total_T1IFN.append(model.total_T1IFN)
        total_TNF.append(model.total_TNF)
        total_IFNg.append(model.total_IFNg)
        total_IL6.append(model.total_IL6)
        total_IL1.append(model.total_IL1)
        total_IL8.append(model.total_IL8)
        total_IL10.append(model.total_IL10)
        total_IL12.append(model.total_IL12)
        total_IL18.append(model.total_IL18)
        total_extracellular_virus.append(model.total_extracellular_virus)
        total_intracellular_virus.append(model.total_intracellular_virus)
        apoptosis_eaten_counter.append(model.apoptosis_eaten_counter)
        infected_epis.append(np.sum(model.epithelium == EpiType.Infected))
        dead_epis.append(np.sum(model.epithelium == EpiType.Dead))
        apoptosed_epis.append(np.sum(model.epithelium == EpiType.Apoptosed))

        model.plot_agents(ax)
        ax.set_title(f"Agents; t={model.time}")
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        ax.set_aspect(1)

        for k, field in enumerate(fields):
            row, col = divmod(k, 4)
            model.plot_field(axs[row, col], field_name=field)
            axs[row, col].set_title(field)
            axs[row, col].get_xaxis().set_visible(False)
            axs[row, col].get_yaxis().set_visible(False)
            axs[row, col].set_aspect(1)
        for k in range(len(fields), 16):
            row, col = divmod(k, 4)
            axs[row, col].set_axis_off()

    pbar.update()
    anim = FuncAnimation(
        fig,
        animate,
        frames=num_steps,
        repeat=False,
        interval=100,  # half the default
    )

In [ ]:
display(pbar.container)

if task == Task.html_anim:
    html_anim = HTML(anim.to_html5_video())
    display(html_anim)
elif task == Task.save_mpeg:
    anim.save("model-run.mpg")
elif task == Task.make_hdf5:
    model.save("test.hdf5")  # t=0
    for _ in range(num_steps):
        model.time_step()
        pbar.update()

        total_T1IFN.append(model.total_T1IFN)
        total_TNF.append(model.total_TNF)
        total_IFNg.append(model.total_IFNg)
        total_IL1.append(model.total_IL1)
        total_IL6.append(model.total_IL6)
        total_IL8.append(model.total_IL8)
        total_IL10.append(model.total_IL10)
        total_IL12.append(model.total_IL12)
        total_IL18.append(model.total_IL18)
        total_extracellular_virus.append(model.total_extracellular_virus)
        total_intracellular_virus.append(model.total_intracellular_virus)
        apoptosis_eaten_counter.append(model.apoptosis_eaten_counter)
        infected_epis.append(np.sum(model.epithelium == EpiType.Infected))
        dead_epis.append(np.sum(model.epithelium == EpiType.Dead))
        apoptosed_epis.append(np.sum(model.epithelium == EpiType.Apoptosed))

        model.save("test.hdf5")

In [ ]:
fig = plt.figure(layout="constrained", figsize=(20, 4))
fig.set_tight_layout(True)

axs = fig.subplots(1, 5)

axs[0].plot(total_extracellular_virus, label="total extracellular virus")
axs[0].plot(total_intracellular_virus, label="total intracellular virus")
axs[0].legend()

# axs[1] epis
axs[1].plot(infected_epis, label="infected epis")
axs[1].plot(dead_epis, label="dead epis")
axs[1].plot(apoptosed_epis, label="apoptosed epis")
axs[1].plot(apoptosis_eaten_counter, label="eaten apoptosed epis")
axs[1].legend()

axs[2].plot(total_TNF, label="TNF")
axs[2].plot(total_IL1, label="IL1")
axs[2].plot(total_IL6, label="IL6")
axs[2].plot(total_IL10, label="IL10")
axs[2].legend()


axs[3].plot(total_T1IFN, label="T1IFN")
axs[3].plot(total_IL18, label="IL18")
axs[3].plot(total_IFNg, label="IFNg")
axs[3].plot(total_IL12, label="IL12")
axs[3].legend()

axs[4].plot(total_IL8, label="IL8")  # not plotted in netlogo version, not sure why
axs[4].legend()